In [1]:
!pip install openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.3/223.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 62.9 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.33.2
    Uninstalling pydantic_core-2.33.2:
      Successfully uninstalled pydantic_core-2.33.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.10
    Uninstalling pydantic-2.11.10:
      Successfully uninstalled pydantic-2.11.10
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
ERROR: pip's dep

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/gdrive


In [3]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [4]:
from agents import Agent, Runner, FileSearchTool, WebSearchTool
import asyncio
import pandas as pd
import numpy as np
from openai import OpenAI
import json
from pydantic import BaseModel

In [69]:
## EDA Agent
## --- Conduct exploratory data analysis based on available data.
## Decide what variable to use and feature engineering.
## --- Available data are in csv format


client = OpenAI()

# class feature_analysis_result(BaseModel):
#   likely_targets:list[str]
#   selected_features:list[str]
#   feature_engineering:list[str]
#   drop_columns:list[str]
#   missing_value_handling:AgentOutputSchema(dict, strict_json_schema=False)

class feature_analysis_Agent:

  '''
  ## Feature Analysis Agent
  ## --- Conduct exploratory data analysis based on available data.
  ## Decide what variable to use and feature engineering.
  ## --- Available data are in csv format

  '''

  def __init__(self, model:str = 'gpt-4o', user_instructions = '', user_defined_target = ''):
    self.model = model
    self.name = 'Feature Analysis Agent'
    self.user_defined_target = user_defined_target
    self.user_instructions = 'You are a data scientist specialized in feature analysis.\
    You analyze data, gives helpful insights on what variable to use and feature engineering.'+user_instructions
    if self.user_defined_target:
      self.user_instructions += f'The target variable is {self.user_defined_target}'
    self.agent = Agent(
        name = self.name,
        model = self.model,
        instructions = self.user_instructions,
        # output_type = feature_analysis_result
        )

  async def run(self, csv_path: str, varb_info_path:str=False):
    df = self._load_csv(csv_path)
    profile = self._profile_data(df)
    suggestions = await self._llm_interpretation(profile, varb_info_path)
    return {
        'raw_profile':profile,
        'Feature_analysis_suggestions':suggestions}

  def _load_csv(self, csv_path:str):
    '''load data'''
    df = pd.read_csv(csv_path)
    return df

  def _profile_data(self, df:pd.DataFrame):
    '''Check dataset profile, such as missing %, dtype, unique counts, etc'''
    profile = {}
    profile['n_rows'] = len(df)
    profile['n_cols'] = len(df.columns)

    ## Check dtype, missing %,
    col_info = df.describe().to_dict()
    for i in df.columns:
      col_info[i]['missing_pct'] = 1 - col_info[i]['count']/len(df)
      col_info[i]['dtype'] = str(df[i].dtype)
      col_info[i]['unique_count'] = len(df[i].unique())

    profile['col_info'] = col_info

    return profile

  async def _llm_interpretation(self, profile_dict, varb_info_path = False):
    '''
    Sends data summary and variable information if any to LLM
    '''
    if varb_info_path:
      with open(varb_info_path, 'r') as file:
        varb_info = file.read()
    else:
      varb_info = 'None'

    prompt = f"""
    Given the following dataset profile:
    {json.dumps(profile_dict, indent = 2)}
    and the variable information below:
    {varb_info}

    Please :
    1. Identify likely target variable if no target provided in instructions; otherwise, use the provided target
    2. Identify useful predictor features
    3. Identify columns to drop and reasoning
    4. Suggest feature engineering (e.g. log transform, bucketization). \
    Any columns identified as 'drop columns' should not be selected for feature engineering.
    5. Summarize missing variable issues and solutions

    Return JSON structured as:
    {{
      'likely_targets':[],
      'selected_features':[],
      'drop_columns':[{{'Feature','Reason'}}],
      'feature_engineering':[{{'Feature','Method','Reason'}}],
      'missing_value_handling':[{{'Feature','Method','Reason'}}]

    }}

    """

    result = await Runner.run(self.agent, prompt)
    print(result.final_output)
    return result.final_output










In [70]:
# ## No target provided
# csv_path = '/content/gdrive/MyDrive/Colab Notebooks/MLAgent/train.csv'
# varb_info_path='/content/gdrive/MyDrive/Colab Notebooks/MLAgent/train_varb_info.txt'
# user_instructions = "build a model that predicts excess returns and includes a betting strategy\
#  designed to outperform the S&P 500 while staying within a 120% volatility constraint. We’ll\
#   provide daily data that combines public market information with our proprietary dataset, giving\
#    you the raw material to uncover patterns most miss."
# fea_agent = feature_analysis_Agent(user_instructions = user_instructions)
# result = await fea_agent.run(csv_path = csv_path, varb_info_path=varb_info_path)


In [71]:
## target provided
csv_path = '/content/drive/MyDrive/Colab Notebooks/MLAgent/train.csv'
varb_info_path='/content/drive/MyDrive/Colab Notebooks/MLAgent/train_varb_info.txt'
user_instructions = "build a model that predicts excess returns and includes a betting strategy\
 designed to outperform the S&P 500 while staying within a 120% volatility constraint. We’ll\
  provide daily data that combines public market information with our proprietary dataset, giving\
   you the raw material to uncover patterns most miss."
fea_agent = feature_analysis_Agent(user_instructions = user_instructions, user_defined_target='market_forward_excess_returns')
result = await fea_agent.run(csv_path = csv_path, varb_info_path=varb_info_path)


```json
{
  "likely_targets": [
    "market_forward_excess_returns"
  ],
  "selected_features": [
    "M*", "E*", "I*", "P*", "V*", "S*", "D*"
  ],
  "drop_columns": [
    {
      "Feature": "date_id",
      "Reason": "Identifier with no predictive power for modeling."
    },
    {
      "Feature": "forward_returns",
      "Reason": "Train set only feature that is not present in test data."
    },
    {
      "Feature": "risk_free_rate",
      "Reason": "Train set only feature that is not present in test data."
    }
  ],
  "feature_engineering": [
    {
      "Feature": "E*",
      "Method": "Normalization",
      "Reason": "Macro Economic features often have a wide range of values and need normalization."
    },
    {
      "Feature": "M*",
      "Method": "Standardization",
      "Reason": "Market dynamics features could benefit from normalization due to varied scales."
    },
    {
      "Feature": "V*",
      "Method": "Log Transform",
      "Reason": "Volatility measures often be

In [72]:
result['Feature_analysis_suggestions']

'```json\n{\n  "likely_targets": [\n    "market_forward_excess_returns"\n  ],\n  "selected_features": [\n    "M*", "E*", "I*", "P*", "V*", "S*", "D*"\n  ],\n  "drop_columns": [\n    {\n      "Feature": "date_id",\n      "Reason": "Identifier with no predictive power for modeling."\n    },\n    {\n      "Feature": "forward_returns",\n      "Reason": "Train set only feature that is not present in test data."\n    },\n    {\n      "Feature": "risk_free_rate",\n      "Reason": "Train set only feature that is not present in test data."\n    }\n  ],\n  "feature_engineering": [\n    {\n      "Feature": "E*",\n      "Method": "Normalization",\n      "Reason": "Macro Economic features often have a wide range of values and need normalization."\n    },\n    {\n      "Feature": "M*",\n      "Method": "Standardization",\n      "Reason": "Market dynamics features could benefit from normalization due to varied scales."\n    },\n    {\n      "Feature": "V*",\n      "Method": "Log Transform",\n      "R

In [73]:
result['raw_profile']['n_rows']

8990

In [83]:
## Build a coding agent to write code according to feature analysis agent's suggestions
class CodingAgent:

  def __init__(self, model:str = 'gpt-4o'):
    self.model = model
    self.name = 'Coding Agent'
    self.user_instructions = 'You are a coding engineer specialized in data science.\
    You take suggestions as instructions from the feature analysis agent and write \
    code to transform the given data. In the end, you return the transformed dataset \
    in a pandas dataframe format. '
    self.agent = Agent(
        name = self.name,
        model = self.model,
        instructions = self.user_instructions,
        )


  async def generate_transformation_code(self, df_profile, suggestions):
    """
    Take suggestions from the feature analysis agent and generate code to transform the data.
    Suggestions from the feature analysis agents are JSON structured as:
    {{
      'likely_targets':[],
      'selected_features':[],
      'feature_engineering':[{{'Feature','Method','Reason'}}],
      'drop_columns':[{{'Feature','Reason'}}],
      'missing_value_handling':[{{'Feature','Method','Reason'}}]

    }}
    """

    prompt = f"""
    Based on the following:
    Data Profile:\n{json.dumps(df_profile, indent = 2)},
    Feature Analysis Agent Suggestions:\n{json.dumps(suggestions, indent = 2)},

    Write python code that:
    1. The dataset is called 'df'. Do not change the name of the dataset. Do not \
    read additional data.
    2. Applies suggested transformations from the Feature Analysis Agent Suggestions
    3. return the transformed data in a pandas dataframe format as 'df_transformed'

    Only return python code. No explanation.
    """

    result = await Runner.run(self.agent, prompt)
    # print(result.final_output)
    return result.final_output

  def execute_code(self, csv_path: str,  code:str):
    '''Executes generated code and return df_transformed'''
    df = pd.read_csv(csv_path)
    local_varbs = {'df':df.copy(), 'pd':pd, 'np': np}
    code = code.replace("```", "")
    code = code.removeprefix("python")
    exec(code, {}, local_varbs)
    df_transformed = local_varbs.get('df_transformed',None)

    return df_transformed

In [84]:
coding_agent = CodingAgent()




In [85]:
code = await coding_agent.generate_transformation_code(
    result['raw_profile'],
    result['Feature_analysis_suggestions']
)

print("Generated code:")
print(code)

Generated code:
```python
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

# Drop specified columns
df = df.drop(['date_id', 'forward_returns', 'risk_free_rate'], axis=1)

# Handle missing values
# Market (M*) - Forward/Backward Fill
df.loc[:, df.columns.str.startswith('M')] = df.loc[:, df.columns.str.startswith('M')].fillna(method='ffill').fillna(method='bfill')

# Economic (E*) - Mean Imputation
mean_imputer = SimpleImputer(strategy='mean')
df.loc[:, df.columns.str.startswith('E')] = mean_imputer.fit_transform(df.loc[:, df.columns.str.startswith('E')])

# Interest (I*) - Interpolation
df.loc[:, df.columns.str.startswith('I')] = df.loc[:, df.columns.str.startswith('I')].interpolate()

# Sentiment (S*) - Mode Imputation
mode_imputer = SimpleImputer(strategy='most_frequent')
df.loc[:, df.columns.str.startswith('S')] = 

In [87]:
# Execute to get transformed df
csv_path = '/content/drive/MyDrive/Colab Notebooks/MLAgent/train.csv'

df_transformed = coding_agent.execute_code(csv_path = csv_path, code = code)

display(df_transformed.head())

<string>:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


,D1,D2,D3,D4,D5,D6,D7,D8,D9,E1,...,V13,V2,V3,V4,V5,V6,V7,V8,V9,market_forward_excess_returns
0,0,0,0,1,1,0,0,0,1,0.274284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.003038
1,0,0,0,1,1,0,0,0,1,0.274284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009114
2,0,0,0,1,0,0,0,0,1,0.274284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.010243
3,0,0,0,1,0,0,0,0,0,0.274284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004046
4,0,0,0,1,0,0,0,0,0,0.274284,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.012301


In [88]:
df = pd.read_csv(csv_path)
display(df.head())

,date_id,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,V3,V4,V5,V6,V7,V8,V9,forward_returns,risk_free_rate,market_forward_excess_returns
0,0,0,0,0,1,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.002421,0.000301,-0.003038
1,1,0,0,0,1,1,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.008495,0.000303,-0.009114
2,2,0,0,0,1,0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.009624,0.000301,-0.010243
3,3,0,0,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004662,0.000299,0.004046
4,4,0,0,0,1,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.011686,0.000299,-0.012301


In [93]:
df_transformed.loc[:, df_transformed.columns.str.startswith('E')].describe()

,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9
count,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000,8990.000000
mean,0.274284,0.504613,0.125290,0.118156,0.068822,0.053882,0.484729,0.630127,0.589076,0.591031,0.117589,0.640725,0.626871,0.635439,0.042167,0.599338,0.121807,0.848626,0.057031,0.262656
std,0.125591,0.317719,0.231714,0.237601,0.107219,0.096837,0.328891,0.132906,0.149832,0.151049,0.071085,0.166182,0.171528,0.157856,0.103128,0.318270,0.209395,0.038960,0.037256,0.281600
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.199897,0.198627,0.004633,0.003309,0.015625,0.016854,0.168432,0.552631,0.493177,0.498959,0.074312,0.517803,0.610592,0.535958,0.006050,0.348776,0.014891,0.848626,0.035268,0.015222
50%,0.274284,0.504613,0.030443,0.015222,0.031250,0.036517,0.484729,0.630127,0.589076,0.591031,0.106538,0.640725,0.646875,0.635439,0.012965,0.599338,0.036400,0.848626,0.055102,0.213600
75%,0.315847,0.799471,0.125290,0.118156,0.076172,0.053882,0.798809,0.718156,0.683444,0.692036,0.133395,0.769044,0.728071,0.751500,0.042167,0.920251,0.121807,0.848626,0.060863,0.427862
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [94]:
df.loc[:, df.columns.str.startswith('E')].describe()

,E1,E10,E11,E12,E13,E14,E15,E16,E17,E18,E19,E2,E20,E3,E4,E5,E6,E7,E8,E9
count,7206.000000,7984.000000,7984.000000,7984.000000,7984.000000,7984.000000,7984.000000,7984.000000,7984.000000,7984.000000,7984.000000,7984.000000,7374.000000,7984.000000,7984.000000,7984.000000,7984.000000,2021.000000,7984.000000,7984.000000
mean,1.564574,0.504941,0.125869,0.118739,0.012314,0.007005,0.485070,-0.047127,-0.007330,0.097865,0.118793,0.500651,0.902917,0.340552,0.016795,0.599603,0.122388,-0.036545,-0.272055,0.263144
std,0.633895,0.336922,0.245719,0.251961,0.019263,0.012097,0.348769,1.137741,1.159378,1.146872,1.247622,1.424679,1.272464,1.506653,0.041870,0.337506,0.222050,1.925521,1.514817,0.298619
min,0.325149,0.000661,0.000661,0.000661,0.000661,0.000661,0.000661,-5.130519,-4.302885,-4.131097,-1.826114,-4.675791,-3.308764,-5.374951,0.000661,0.000661,0.000661,-19.918972,-2.457316,0.000661
25%,1.114633,0.165675,0.004960,0.003638,0.002976,0.002315,0.127976,-0.766907,-0.830320,-0.659848,-0.642287,-0.560370,0.440873,-0.661601,0.002646,0.305556,0.013228,-0.124696,-1.185839,0.007606
50%,1.516651,0.505787,0.019511,0.006944,0.005291,0.004299,0.441303,0.001765,0.062710,0.003939,-0.203393,0.693642,1.220291,0.308699,0.004960,0.650132,0.030754,0.016171,-0.514610,0.104167
75%,1.876977,0.822751,0.087384,0.074818,0.015212,0.006283,0.833333,0.763389,0.808831,0.909992,0.507321,1.638542,1.711411,1.482888,0.007275,0.953125,0.098545,0.425972,0.022738,0.471974
max,4.843911,1.000000,1.000000,1.000000,0.169974,0.118386,1.000000,2.936733,2.989140,3.024136,14.713768,3.403242,3.409812,3.619623,0.383267,1.000000,1.000000,3.509994,35.860072,1.000000
